In [559]:
import pandas as pd
import sys
sys.path.insert(1, r'C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge')
from utils import *
import pickle
# catboosting
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb

In [613]:
df = pd.read_csv('../data/consumo_material_clean_with_category.csv', parse_dates=['FECHAPEDIDO'])
# df_train = pd.read_csv('../data/consumo_no23.csv')
# df_test = pd.read_csv('../data/consumo_23.csv')

df_transito = df[df['TGL'] == 'TRANSITO']
df_alma = df[df['TGL'] == 'ALMACENABLE']

specified_ids = ['E65159', 'E64751', 'E64764']
df_specified_ids = df_alma[df_alma['CODIGO'].isin(specified_ids)]
df_other_ids = df_alma[~df_alma['CODIGO'].isin(specified_ids)]
df_other_ids_gt_10 = df_other_ids[df_other_ids['UNIDADESCONSUMOCONTENIDAS'] > 10]
df_other_ids_eq_10 = df_other_ids[df_other_ids['UNIDADESCONSUMOCONTENIDAS'] == 10]
df_other_ids_lt_10 = df_other_ids[df_other_ids['UNIDADESCONSUMOCONTENIDAS'] < 10]

In [ ]:
df_other_ids[df_other_ids['STACKS_COMPRATS'] > 100]['UNIDADESCONSUMOCONTENIDAS'].unique()

In [ ]:
df_other_ids[df_other_ids['STACKS_COMPRATS'] < 100]['UNIDADESCONSUMOCONTENIDAS'].unique()

In [614]:
df = preprocess_cosumo(df_other_ids_lt_10)
df.head()

asjfkjdsfljds


C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge\utils.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['STACKS_COMPRATS'] = data['CANTIDADCOMPRA']/data['UNIDADESCONSUMOCONTENIDAS']
C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge\utils.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['STACKS_COMPRATS'] = data['STACKS_COMPRATS'].apply(lambda x: int(x))
C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge\utils.py:12: SettingWithCopyWarn

,FECHAPEDIDO,UNIDADESCONSUMOCONTENIDAS,PRECIO,STACKS_COMPRATS,CODIGO_E64663,CODIGO_E64932,CODIGO_E65056,CODIGO_E65485,CODIGO_E70130,TIPOCOMPRA_Compra menor,TIPOCOMPRA_Concurso
11,2018-01-02,3,1.851300,294,0,0,1,0,0,1,0
18,2021-01-02,5,26.399925,13,1,0,0,0,0,0,1
39,2023-01-02,3,0.989571,91,0,0,1,0,0,0,1
50,2016-01-03,5,29.150000,50,1,0,0,0,0,1,0
53,2016-01-03,3,1.851300,300,0,0,1,0,0,1,0


In [615]:
df['trimester'] = df['FECHAPEDIDO'].dt.quarter
#df['month'] = df['FECHAPEDIDO'].dt.month

In [616]:
grouped_df = df.groupby(['trimester', df['FECHAPEDIDO'].dt.year])

In [617]:
len(grouped_df)

36

In [648]:
trimester_dfs = [grouped_df.get_group((trimester, year)).reset_index(drop=True) for (trimester, year), _ in grouped_df]
dfs_chunked = [trimester_dfs[i:i + 9] for i in range(0, len(trimester_dfs), 9)]
result_dfs = []
for chunk in dfs_chunked:
    result_dfs.append(pd.concat(chunk, axis=0, ignore_index=True))

In [649]:
len(result_dfs)

4

In [651]:
target = result_dfs[3]

In [652]:
target.drop(['trimester'], axis = 1, inplace = True)
target.head()

,FECHAPEDIDO,UNIDADESCONSUMOCONTENIDAS,PRECIO,STACKS_COMPRATS,CODIGO_E64663,CODIGO_E64932,CODIGO_E65056,CODIGO_E65485,CODIGO_E70130,TIPOCOMPRA_Compra menor,TIPOCOMPRA_Concurso
0,2015-10-02,5,8.250039,75,0,0,0,1,0,1,0
1,2015-10-03,5,5.245370,51,0,1,0,0,0,1,0
2,2015-10-09,5,5.245370,7,0,1,0,0,0,1,0
3,2015-10-09,5,14.850000,13,0,0,0,0,1,1,0
4,2015-10-09,5,14.850000,10,0,0,0,0,1,1,0


In [653]:
df_train = target[target['FECHAPEDIDO']<dt.datetime(year=2023,month=1,day=1)]
#df_train = df_train[df_train['FECHAPEDIDO'] >= dt.datetime(year=2021,month=1,day=1)]
df_test = target[target['FECHAPEDIDO']>=dt.datetime(year=2023,month=1,day=1)]

df_train.drop(columns=['FECHAPEDIDO'], inplace=True)
df_test.drop(columns=['FECHAPEDIDO'], inplace=True)

C:\Users\adars\AppData\Local\Temp\ipykernel_14272\1363158763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=['FECHAPEDIDO'], inplace=True)
C:\Users\adars\AppData\Local\Temp\ipykernel_14272\1363158763.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns=['FECHAPEDIDO'], inplace=True)


In [654]:
print(df_train.shape)
print(df_test.shape)

(439, 10)
(20, 10)


In [655]:
X = df_train.drop(['STACKS_COMPRATS'], axis = 1)
y = df_train['STACKS_COMPRATS']
X_test = df_test.drop(['STACKS_COMPRATS'], axis = 1)
y_test = df_test['STACKS_COMPRATS'].reset_index(drop=True)

In [656]:
X.columns

Index(['UNIDADESCONSUMOCONTENIDAS', 'PRECIO', 'CODIGO_E64663', 'CODIGO_E64932',
       'CODIGO_E65056', 'CODIGO_E65485', 'CODIGO_E70130',
       'TIPOCOMPRA_Compra menor', 'TIPOCOMPRA_Concurso'],
      dtype='object')

In [657]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', subsample = 1, reg_lambda = 0.2, reg_alpha = 0.1, colample_bytree = 0.2, learning_rate = 0.1,
                max_depth = 2, alpha = 100, n_estimators = 1000)
xg_reg.fit(X, y)

[07:07:55] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[07:07:55] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "colample_bytree" } are not used.



XGBRegressor(alpha=100, base_score=None, booster=None, callbacks=None,
             colample_bytree=0.2, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None, ...)

In [658]:
# Predict the inventory units for the test and train sets using the trained CatBoostRegressor model 'gsc1'
y_predicted_test = xg_reg.predict(X_test)
y_predicted_train = xg_reg.predict(X)

# Calculate the root mean squared error (RMSE) between the predicted and actual inventory units for the test and train sets
# The 'squared' parameter is set to False to return the RMSE instead of the MSE
rms_test = mean_squared_error(y_test, y_predicted_test)
rms_train = mean_squared_error(y, y_predicted_train)

# Print the RMSE values for the test and train sets
print(f'train: {rms_train}\ntest: {rms_test}')

train: 3880.8497112826103
test: 14866.817992906184


In [647]:
with open('../trained_models/xgb_other_ids_lt_10_4Q.pkl', 'wb') as file:
    pickle.dump(xg_reg, file)